<a href="https://github.com/ternaustralia/cmrset-examples/blob/main/gee-examples/python/Example_2_Aggregating_CMRSET_for_a_set_of_Features_(Spatial_Aggregation).ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View on GitHub"/></a>
&nbsp;
<a href="https://colab.research.google.com/github/ternaustralia/cmrset-examples/blob/main/gee-examples/python/Example_2_Aggregating_CMRSET_for_a_set_of_Features_(Spatial_Aggregation).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
&nbsp;
<a href="https://mybinder.org/v2/gh/ternaustralia/cmrset-examples/HEAD?labpath=gee-examples%2Fpython%2FExample_2_Aggregating_CMRSET_for_a_set_of_Features_(Spatial_Aggregation).ipynb" target="_parent"><img src="https://mybinder.org/badge_logo.svg" alt="Open In Binder"/></a>


# Example 2 - Aggregating CMRSET for a set of Features (Spatial Aggregation)
 This example illustrates how to compute the average evapotranspiration for a set of features (such as a multi-polygon/multi-point set). In this example we will use the Australian World Herritage Sites (which cover land) as our feature set. The monthly timeseries for each feature is then exported as a CSV to Google Drive.

To run this example it is assumed you have [registered](https://signup.earthengine.google.com/) to [Google Earth Engine](https://earthengine.google.com/). Please feel free to adapt this example to meet your own requirements.

---
**Execution Time:** ~8 minutes

**Storage Required:** 16 kB (on Google Drive)

## Install geemap
The [geemap](https://geemap.org/) Python package is built upon [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) and [folium](https://github.com/python-visualization/folium) packages, and enables users to analyze and visualize Earth Engine datasets interactively within a Jupyter-based environment.  The geemap API also mimics the official Google Earth Engine Javascript Map API when making calls such as `Map.addLayer()`, `Map.setCenter()`, and `Map.centerObject()`, etc. This simplifies the translation between both languages if you are transitioning from Javascript to Python.

Note: the package ipyleaflet is not compatible with Google Colab, hence you will notice we specifically import the [folium implementation](https://geemap.org/foliumap/) in this example for broarder compatibility.

In [6]:
# Installs the geemap package
import subprocess

try:
    import geemap.foliumap as geemap
except ImportError:
    print('Installing geemap...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

In [7]:
import geemap.foliumap as geemap

# Authenticate and initialize
Run the `ee.Authenticate()` function to authenticate your access to Earth Engine servers and `ee.Initialize()` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. **Please follow the instructions in the prompts.** You will also need to enter your Google Earth Engine Cloud Project ID below.


In [8]:
# Import the Earth Engine module
import ee

# Cloud Project ID to associate with your Earth Engine workspace.
project = "Your Project ID to use"

try:
    ee.Initialize(project=project) # Initialize the library.
except Exception as e:
    ee.Authenticate() # If initialization fails, authentication is required... Trigger the authentication flow and initialize again.
    ee.Initialize(project=project) # Initialize the library.

## Compute the aggregated timeseries for each feature
Firstly, let's define the regions of interest. We will use the [World Database on Protected Areas](https://developers.google.com/earth-engine/datasets/catalog/WCMC_WDPA_current_polygons) dataset (WDPA), then [filter](https://developers.google.com/earth-engine/guides/feature_collection_filtering) the [Feature Collection](https://developers.google.com/earth-engine/guides/feature_collections) upon each [Features](https://developers.google.com/earth-engine/guides/features) attributes so we only use Australian herritage sites covering land.

To use your own Feature Collection, you can [upload](https://developers.google.com/earth-engine/guides/table_upload) your features to your Google Earth Engine Account and adjust the identifier referenced below.  This example will work with any *type* of feature data in your Feature Collection (polygons, points, transects... etc).

In [9]:
# Define the regions of interest.
regions = ee.FeatureCollection("WCMC/WDPA/current/polygons") \
          .filterMetadata('DESIG', 'equals', 'World Heritage Site (natural or mixed)') \
          .filterMetadata('ISO3', 'equals', 'AUS')  \
          .filterMetadata('MARINE', 'equals', '0')

print("Number of features: {0}".format(regions.size().getInfo()))

Number of features: 10


Now, let's [visualize our features](https://developers.google.com/earth-engine/guides/feature_collections_visualizing) on the map...

In [10]:
# Visualize the regions of interest.
Map = geemap.Map(add_google_map = False)
Map.addLayer(regions, {'color': 'green'}, 'Study Sites')
Map.centerObject(regions)
Map

Next, we will access [CMRSET](https://developers.google.com/earth-engine/datasets/catalog/TERN_AET_CMRSET_LANDSAT_V2_2) from the [Earth Engine data catalog](https://developers.google.com/earth-engine/datasets), and filter down the [ImageCollection](https://developers.google.com/earth-engine/tutorials/tutorial_api_04) to the period of interest.

In [11]:
# Define the period of interest.
start = '2012-02-01'
end = '2021-03-01'

# Access evapotranspiration from the CMRSET dataset.
cmrset_aet = ee.ImageCollection("TERN/AET/CMRSET_LANDSAT_V2_2").filterDate(start, end).select(['ETa'])

Lastly, we will obtain the [statistics of our regions](https://developers.google.com/earth-engine/guides/reducers_reduce_regions) for each image. Here we will use a `mean()` reducer to compute the average of the pixels for each feature... since the average statistician is just plain mean? 😮 However if you wish to use a different statistic you can refer to *ee.Reducer* in the **Client Libraries** [API Reference](https://developers.google.com/earth-engine/apidocs).

In [12]:
# A helper function to assist with aggregating the features.
def reduceFeaturesForImage(image):

  # Calculate average evapotranspiration for each feature in the set.
  mean_aet = image.reduceRegions(**{
    'collection': regions,
    'reducer': ee.Reducer.mean(),
    'scale': 30
  })

  # Transfer the image date across to each feature as a new attribute.
  date = image.date().format('yyyy-MM-dd')
  return mean_aet.map(lambda feature: feature.set('Date', date))

# For each image, compute the aggregate for each feature.
reduced_aet = cmrset_aet.map(reduceFeaturesForImage).flatten()

## Export the CSV to Google Drive
Before we export, let's first take a look at the [properties](https://developers.google.com/earth-engine/guides/feature_collection_info) (sometimes also known as *attributes* or *metadata*) which are associated with each feature so we can decide what data to export. We will just inspect the first feature...

In [ ]:
# Get the properties for the first feature.
properties = reduced_aet.first().toDictionary().getInfo()

# Print each properties key/value pair.
for property in properties:
  print('{0}: {1}'.format(property, properties[property]))

Now, let's define an [export task](https://developers.google.com/earth-engine/guides/exporting) from Earth Engine which will [export the table to Google Drive](https://developers.google.com/earth-engine/apidocs/export-table-todrive) as a CSV file...  After observing the properties we have available, let's select the following properties as columns for our CSV file to be exported:


1.   **NAME**: The name of the protected region (property already available on our features of interest).
2.   **Date**: The image date which was transferred to each feature during the aggregation process (poperty created within this script).
3.   **mean**: The resulting property which was generated after running the  `mean()` reducer (property created within this script).

We will also *sort* our features by the **NAME** property so our CSV records will be grouped by each site during the export.

In [ ]:
# Define the export settings.
export_folder = 'CMRSET_Exports'
task_description = 'CMRSET_{0}_{1}_Monthly'.format(start, end)

# Export the table, specifying the columns to be included.
task = ee.batch.Export.table.toDrive(**{
    'collection': reduced_aet.sort('NAME'), # Sort the records by NAME property.
    'selectors': ['NAME', 'Date', 'mean'],  # Select these properties to be exported.
    'description': task_description,
    'folder': export_folder
})
task.start()

Now that your task has started, you may monitor it's progress under the **Tasks** tab in the [Earth Engine Code Editor](https://code.earthengine.google.com/) or via the [Earth Engine Task Manager](https://code.earthengine.google.com/tasks)